# FreeCAD 파라메트릭 스케치 생성 튜토리얼

이 튜토리얼은 FreeCAD Python API를 사용하여 파라메트릭 2D 스케치를 생성하는 방법을 단계별로 설명합니다.
FreeCAD의 GUI 없이도 프로그래밍 방식으로 정확한 기하학적 도형을 만들 수 있습니다.

## 📚 학습 목표
- FreeCAD Python API 기본 사용법
- 2D 스케치 생성 및 도형 추가
- 구속 조건(Constraints) 적용
- 파라메트릭 모델링의 기본 개념

## 🔧 필요한 사전 지식
- Python 기본 문법
- 기본적인 기하학 개념 (점, 선, 원, 각도 등)

## 1단계: FreeCAD 환경 설정

### 🔍 FreeCAD 설치 경로 확인

먼저 사용자의 시스템에 맞는 FreeCAD 설치 경로를 설정해야 합니다.

**운영체제별 일반적인 FreeCAD 설치 경로:**
- **macOS**: `/Applications/FreeCAD.app/Contents/Resources/lib`
- **Windows**: `다운로드 받은 패키지 파일의 bin 폴더 경로`

> ⚠️ **중요**: 아래 `FREECADPATH` 변수를 본인의 FreeCAD 설치 경로에 맞게 수정하세요!

In [ ]:
import sys
import os

# FreeCAD 설치 경로를 Python 경로에 추가합니다.
# 사용자의 FreeCAD 설치 경로에 맞게 수정해주세요!
FREECADPATH = "/Applications/FreeCAD.app/Contents/Resources/lib"  # macOS 예시
sys.path.append(FREECADPATH)

print(f"FreeCAD 경로 추가됨: {FREECADPATH}")
print(f"Python 경로 개수: {len(sys.path)}")

### 📦 FreeCAD 모듈 임포트

FreeCAD의 핵심 모듈들을 임포트합니다. GUI 모듈은 제외하고 필요한 모듈만 가져옵니다.

In [ ]:
# FreeCAD 모듈들을 임포트합니다.
try:
    import FreeCAD as App  # FreeCAD의 핵심 애플리케이션 모듈
    import Part           # 3D 형상 및 기하학적 객체를 다루는 모듈
    import Sketcher       # 2D 스케치 및 구속 조건을 다루는 모듈
    
    print("✅ FreeCAD 모듈이 성공적으로 로드되었습니다.")
    print(f"   - FreeCAD 버전: {App.Version()}")
    print(f"   - 사용 가능한 모듈: App, Part, Sketcher")
    
except ImportError as e:
    print(f"❌ FreeCAD 모듈 로드 실패: {e}")
    print("\n🔧 문제 해결 방법:")
    print("   1. FREECADPATH 경로를 확인하세요")
    print("   2. FreeCAD가 올바르게 설치되었는지 확인하세요")
    print("   3. Python 버전이 FreeCAD와 호환되는지 확인하세요")
    raise

## 2단계: 기본 도형 생성 함수들

FreeCAD에서 사용할 기본 도형들을 생성하는 함수들을 정의합니다.
각 함수는 2D 좌표를 받아서 FreeCAD의 3D 객체로 변환합니다.

In [ ]:
def draw_line(start_point, end_point):
    """
    두 점을 연결하는 직선을 생성합니다.
    
    매개변수:
        start_point (tuple): 시작점의 (x, y) 좌표
        end_point (tuple): 끝점의 (x, y) 좌표
    
    반환값:
        Part.LineSegment: FreeCAD 선분 객체
    """
    # 2D 좌표를 3D 벡터로 변환 (z=0으로 설정)
    start_3d = App.Vector(start_point[0], start_point[1], 0)
    end_3d = App.Vector(end_point[0], end_point[1], 0)
    
    # FreeCAD LineSegment 객체 생성
    return Part.LineSegment(start_3d, end_3d)

def draw_circle(center, radius):
    """
    원을 생성합니다.
    
    매개변수:
        center (tuple): 원의 중심점 (x, y) 좌표
        radius (float): 원의 반지름
    
    반환값:
        Part.Circle: FreeCAD 원 객체
    """
    # 2D 좌표를 3D 벡터로 변환
    center_3d = App.Vector(center[0], center[1], 0)
    
    # FreeCAD Circle 객체 생성
    # 매개변수: 중심점, 법선벡터(z축 방향), 반지름
    return Part.Circle(center_3d, App.Vector(0, 0, 1), radius)

def draw_arc(start_point, end_point, mid_point):
    """
    세 점을 지나는 원호를 생성합니다.
    
    매개변수:
        start_point (tuple): 호의 시작점 (x, y)
        end_point (tuple): 호의 끝점 (x, y)
        mid_point (tuple): 호 위의 중간점 (x, y) - 호의 형태를 결정
    
    반환값:
        Part.Arc: FreeCAD 호 객체
    """
    # 2D 좌표들을 3D 벡터로 변환
    start_3d = App.Vector(start_point[0], start_point[1], 0)
    end_3d = App.Vector(end_point[0], end_point[1], 0)
    mid_3d = App.Vector(mid_point[0], mid_point[1], 0)
    
    # FreeCAD Arc 객체 생성 (시작점, 중간점, 끝점 순서)
    return Part.Arc(start_3d, mid_3d, end_3d)

print("✅ 기본 도형 생성 함수들이 정의되었습니다.")

## 3단계: 파라메트릭 스케치 생성

### 🎯 파라메트릭 모델링이란?

파라메트릭 모델링은 다음과 같은 특징을 가집니다:
- **치수와 관계**를 통해 형상을 정의하는 방법
- **구속 조건(Constraints)**을 통해 도형 간의 관계를 설정
- 하나의 치수를 변경하면 관련된 모든 형상이 **자동으로 업데이트**

### 📐 이 예제에서 생성할 스케치
- 직사각형 모양의 기본 윤곽 (3개의 직선 + 1개의 호)
- 중앙에 위치한 원
- 모든 도형이 구속 조건으로 연결됨

In [ ]:
# 새로운 FreeCAD 문서를 생성합니다.
doc = App.newDocument("ParametricSketchTutorial")

print(f"✅ 새 문서 생성됨: {doc.Name}")
print(f"   문서 라벨: {doc.Label}")

In [ ]:
# XY 평면에 새로운 스케치 객체를 추가합니다.
sketch = doc.addObject('Sketcher::SketchObject', 'TutorialSketch')

print(f"✅ 스케치 객체 생성됨: {sketch.Name}")
print(f"   스케치 라벨: {sketch.Label}")
print(f"ℹ️  스케치는 XY 평면(z=0)에 생성됩니다.")

### 🔷 기하학적 도형 추가

`addGeometry()` 메서드는 각 도형의 내부 인덱스 번호를 반환합니다.
이 인덱스는 나중에 구속 조건을 적용할 때 사용됩니다.

In [ ]:
print("🔷 기하학적 도형 추가를 시작합니다...")
print("\n1️⃣ 직사각형의 기본 윤곽선 생성")

# 하단 수평선: (0,0) → (80,0)
idx_line1 = sketch.addGeometry(draw_line((0, 0), (80, 0)))
print(f"   ✅ 선분 1 (하단): 인덱스 {idx_line1}, (0,0) → (80,0)")

# 우측 수직선: (80,0) → (80,50)
idx_line2 = sketch.addGeometry(draw_line((80, 0), (80, 50)))
print(f"   ✅ 선분 2 (우측): 인덱스 {idx_line2}, (80,0) → (80,50)")

# 좌측 수직선: (0,50) → (0,0)
idx_line3 = sketch.addGeometry(draw_line((0, 50), (0, 0)))
print(f"   ✅ 선분 3 (좌측): 인덱스 {idx_line3}, (0,50) → (0,0)")

print(f"\n현재 스케치의 도형 개수: {len(sketch.Geometry)}개")

In [ ]:
print("2️⃣ 상단 호 생성")

# 호의 시작점과 끝점
start_point_arc = (80, 50)  # 우측 선분의 끝점
end_point_arc = (0, 50)     # 좌측 선분의 시작점

# 호의 중간점 (호의 모양을 결정하는 점)
# x 중심에서 위쪽으로 올려서 반원 모양을 만듭니다.
mid_point_on_arc = (40, 90)  # 중심 x=40, 위쪽으로 40만큼 올림

idx_arc = sketch.addGeometry(draw_arc(start_point_arc, end_point_arc, mid_point_on_arc))
print(f"   ✅ 호: 인덱스 {idx_arc}, (80,50) → (0,50), 중간점 (40,90)")

print(f"\n현재 스케치의 도형 개수: {len(sketch.Geometry)}개")

In [ ]:
print("3️⃣ 중앙 원 생성")

center_point = (40, 25)  # 직사각형의 중심 근처
circle_radius = 15       # 적당한 크기의 반지름

idx_circle = sketch.addGeometry(draw_circle(center_point, circle_radius))
print(f"   ✅ 원: 인덱스 {idx_circle}, 중심 (40,25), 반지름 {circle_radius}")

print(f"\n✅ 모든 도형 추가 완료!")
print(f"   총 도형 개수: {len(sketch.Geometry)}개")
print(f"   도형 인덱스: 선분들({idx_line1}, {idx_line2}, {idx_line3}), 호({idx_arc}), 원({idx_circle})")

### 🔗 구속 조건(Constraints) 추가

구속 조건은 도형들 간의 관계를 정의하여 스케치를 완전하게 만듭니다.

**FreeCAD에서 점 인덱스:**
- `1` = 시작점
- `2` = 끝점

In [ ]:
print("🔗 구속 조건 적용을 시작합니다...")
print("\n1️⃣ 점 일치 구속 조건 추가")
print("   (도형의 끝점들을 연결하여 닫힌 윤곽을 만듭니다)")

# 선분1의 끝점과 선분2의 시작점을 연결
sketch.addConstraint(Sketcher.Constraint('Coincident', idx_line1, 2, idx_line2, 1))
print("   ✅ 선분1 끝점 ↔ 선분2 시작점")

# 선분2의 끝점과 호의 시작점을 연결
sketch.addConstraint(Sketcher.Constraint('Coincident', idx_line2, 2, idx_arc, 1))
print("   ✅ 선분2 끝점 ↔ 호 시작점")

# 호의 끝점과 선분3의 시작점을 연결
sketch.addConstraint(Sketcher.Constraint('Coincident', idx_arc, 2, idx_line3, 1))
print("   ✅ 호 끝점 ↔ 선분3 시작점")

# 선분3의 끝점과 선분1의 시작점을 연결 (윤곽 완성)
sketch.addConstraint(Sketcher.Constraint('Coincident', idx_line3, 2, idx_line1, 1))
print("   ✅ 선분3 끝점 ↔ 선분1 시작점")

print(f"\n현재 구속 조건 개수: {len(sketch.Constraints)}개")

In [ ]:
print("2️⃣ 방향 구속 조건 추가")
print("   (선분들의 방향을 고정하여 형태를 안정화합니다)")

# 수평 구속: 선분을 수평으로 고정
sketch.addConstraint(Sketcher.Constraint('Horizontal', idx_line1))
print("   ✅ 선분1을 수평으로 고정")

# 수직 구속: 선분을 수직으로 고정
sketch.addConstraint(Sketcher.Constraint('Vertical', idx_line2))
print("   ✅ 선분2를 수직으로 고정")

sketch.addConstraint(Sketcher.Constraint('Vertical', idx_line3))
print("   ✅ 선분3을 수직으로 고정")

print(f"\n✅ 모든 구속 조건 추가 완료!")
print(f"   총 구속 조건 개수: {len(sketch.Constraints)}개")

### 🔄 문서 재계산 및 완료

In [ ]:
# FreeCAD 문서를 재계산하여 모든 변경사항을 적용합니다.
doc.recompute()

print("✅ 문서 재계산 완료")
print("🎉 파라메트릭 스케치 생성 완료!")

# 스케치 정보 요약
print(f"\n📊 생성된 스케치 정보:")
print(f"   - 문서명: {doc.Name}")
print(f"   - 스케치명: {sketch.Name}")
print(f"   - 도형 개수: {len(sketch.Geometry)}개")
print(f"   - 구속 조건 개수: {len(sketch.Constraints)}개")

# 각 도형의 상세 정보
print(f"\n🔍 도형 상세 정보:")
for i, geom in enumerate(sketch.Geometry):
    print(f"   [{i}] {type(geom).__name__}: {geom}")

## 4단계: 실행 및 결과 저장

생성된 스케치를 FreeCAD 파일 형식으로 저장합니다.

In [ ]:
# 출력 파일명 설정
output_filename = "parametric_sketch_tutorial_result.FCStd"
output_path = os.path.join(os.getcwd(), output_filename)

try:
    # FreeCAD 문서를 파일로 저장
    doc.saveAs(output_path)
    
    print(f"✅ 스케치가 성공적으로 생성되고 저장되었습니다!")
    print(f"📁 저장 위치: {output_path}")
    print(f"📏 파일 크기: {os.path.getsize(output_path)} bytes")
    
except Exception as e:
    print(f"❌ 파일 저장 중 오류 발생: {e}")

## 💡 다음 단계 가이드

1. **FreeCAD를 실행**하세요
2. **파일 → 열기** → `parametric_sketch_tutorial_result.FCStd` 선택
3. 생성된 스케치를 확인하고 편집해보세요
4. 스케치를 더블클릭하여 편집 모드로 진입
5. 구속 조건을 추가하거나 수정해보세요

## 🎓 추가 학습 아이디어

- **치수 구속 조건** 추가해보기 (길이, 각도 등)
- **대칭 구속 조건** 적용해보기
- **다른 도형들** 추가해보기 (타원, 스플라인 등)
- **스케치를 3D로 돌출(Extrude)**해보기

## 📚 참고 자료

- [FreeCAD 공식 문서](https://wiki.freecadweb.org/Python_scripting_tutorial)
- [FreeCAD Python API 레퍼런스](https://freecad.github.io/SourceDoc/)
- [스케치 구속 조건 가이드](https://wiki.freecadweb.org/Sketcher_Workbench)

즐거운 FreeCAD 학습 되세요! 🛠️